## CNN

In [1]:
# Add Lincoln to system path
import sys
sys.path.append("/Users/seth/development/lincoln/")

In [2]:
from torch import Tensor
import torch

import typing
from typing import List, Tuple

**Note:** Old losses and layers code.

In [3]:
import lincoln as lnc
from lincoln.layers import Dense, Conv2D, Layer
from lincoln.losses import LogLoss, LogSigmoidLoss, MeanSquaredError
from lincoln.optimizers import SGD
from lincoln.operations import ReLU, Softmax
from lincoln.activations import Sigmoid, LogSigmoid
from lincoln.network import NeuralNetwork

In [4]:
from torchvision.datasets import MNIST
mnist_trainset = MNIST(root="./explanatory/data", train=True, download=False, transform=None)
mnist_testset = MNIST(root="./explanatory/data", train=False, download=True, transform=None)

#### One hot encode labels

In [5]:
data = mnist_trainset
num_labels = len(data.train_labels)
train_labels = torch.zeros(num_labels, 10)
for i in range(num_labels):
    train_labels[i][data.train_labels[i]] = 1
train_labels

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]])

In [6]:
data = mnist_testset
num_labels = len(data.test_labels)
test_labels = torch.zeros(num_labels, 10)
for i in range(num_labels):
    test_labels[i][data.test_labels[i]] = 1
test_labels

tensor([[0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [7]:
mnist_train = mnist_trainset.train_data.type(torch.float32).unsqueeze(3) / 255.0
mnist_test = mnist_testset.test_data.type(torch.float32).unsqueeze(3) / 255.0

In [8]:
X_train = mnist_train
X_test = mnist_test
y_train = train_labels
y_test = test_labels

In [9]:
cython = True

In [10]:
def _conv_transform(input_: Tensor) -> Tensor:

    return input_.view(input_.shape[0],
                       input_.shape[1] * input_.shape[2] * input_.shape[3])

X_train_flat = _conv_transform(X_train)
X_test_flat = _conv_transform(X_test)

In [15]:
conv_nn_ll = NeuralNetwork(
    layers=[Dense(neurons=250, 
                  activation=Sigmoid()),
            Dense(neurons=50, 
                  activation=Sigmoid()),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogLoss(),
            optimizer = SGD(0.1))

In [16]:
import time
start = time.time()
conv_nn_ll.fit(X_train_flat, y_train, X_test_flat, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=100,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")

Validation loss after 1 epochs is 23464.492
Training took 3.7 seconds.


In [ ]:
def eval_nn(nn, X_test, y_test):
    test_preds = X_test.forward(y_test)
    
    test_preds_l = torch.argmax(test_preds, dim=1)
    
    test_vals_l = torch.argmax(y_test, dim=1)

In [17]:
eval_nn(conv_nn_ll, X_test_flat, y_test)

NameError: name 'eval_nn' is not defined

In [ ]:
conv_nn_logloss = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=24,
                   activation=ReLU(),
                   cython=cython),
            Conv2D(param_size=3,
                   out_channels=30,
                   activation=ReLU(),
                   cython=cython,
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogLoss(),
            optimizer = SGD())

In [ ]:
import time
start = time.time()
conv_nn_logloss.fit(X_train, y_train, X_test, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=100,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")